In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

# Define the path to your PDF files
pdf_dir = "./docs"  # Cambia esto al directorio donde tienes tus PDFs

# Load PDF documents
pdf_files = [os.path.join(pdf_dir, file) for file in os.listdir(pdf_dir) if file.endswith(".pdf")]
docs = [PyPDFLoader(pdf).load() for pdf in pdf_files]

# Convert list of lists to a single list of documents
docs_list = [item for sublist in docs for item in sublist]

In [2]:
docs_list

[Document(metadata={'source': './docs/Politica Atencion Cliente.pdf', 'page': 0}, page_content='Política de Atención al Cliente - T echnoV erde\nS.A.\n1. Introducción\nT echnoV erde S.A. es una empresa líder en soluciones tecnológicas sostenibles\npara hogares y empresas. Nuestro compromiso con la excelencia en el servicio\nal cliente es fundamental para nuestra misión de crear un futuro más verde\ny eficiente. Esta política establece los principios y procedimientos que guían\nnuestras interacciones con los clientes, asegurando una experiencia excepcional\nen cada punto de contacto.\n2. Objetivos\n• Proporcionar un servicio al cliente de clase mundial que supere las expec-\ntativas.\n• Resolver las inquietudes y problemas de los clientes de manera eficiente y\nefectiva.\n• F omentar la lealtad del cliente a través de interacciones positivas y solu-\nciones personalizadas.\n• Mantener una comunicación clara y transparente en todos los canales de\natención.\n3. Canales de Atención al Cli

In [3]:
# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=100)
doc_splits = text_splitter.split_documents(docs_list) 

In [4]:
len(doc_splits)

105

In [5]:
from langchain_community.vectorstores import Chroma

# Define el directorio donde se almacenará la base de datos
persist_directory = "./chroma_db"  # Puedes cambiar esto al directorio que prefieras

# Crear la base de datos de vectores local
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=OpenAIEmbeddings(),
    persist_directory=persist_directory  # Especifica el directorio de persistencia
)

# # Cargar la base de datos de vectores desde el directorio local
# vectorstore = Chroma(
#     collection_name="rag-chroma",
#     embedding=OpenAIEmbeddings(),
#     persist_directory=persist_directory  # Especifica el directorio de persistencia
# )

In [6]:
retriever = vectorstore.as_retriever()

from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_company_docs",
    "Search and return information about the company documents",
)

In [8]:
question = "Como se llama la empresa?"
documents = retriever.invoke(question)
documents

[Document(metadata={'page': 0, 'source': './docs/Politica Atencion Cliente.pdf'}, page_content='S.A.\n1. Introducción\nT echnoV erde S.A. es una empresa líder en soluciones tecnológicas sostenibles\npara hogares y empresas. Nuestro compromiso con la excelencia en el servicio'),
 Document(metadata={'page': 0, 'source': './docs/Politica Atencion Cliente.pdf'}, page_content='para hogares y empresas. Nuestro compromiso con la excelencia en el servicio\nal cliente es fundamental para nuestra misión de crear un futuro más verde'),
 Document(metadata={'page': 0, 'source': './docs/Politica Atencion Cliente.pdf'}, page_content='Política de Atención al Cliente - T echnoV erde\nS.A.\n1. Introducción\nT echnoV erde S.A. es una empresa líder en soluciones tecnológicas sostenibles'),
 Document(metadata={'page': 3, 'source': './docs/Politica Atencion Cliente.pdf'}, page_content='T odos los nuevos empleados de atención al cliente reciben un programa de forma-\nción intensivo de 4 semanas que cubre: - 

In [10]:
# Cargar la base de datos de vectores desde el directorio local
persist_directory = "./chroma_db"
vectorstore = Chroma(
    collection_name="rag-chroma",
    persist_directory=persist_directory  # Especifica el directorio de persistencia
)